In [ ]:
!wget https://raw.githubusercontent.com/bigmlcom/python/master/data/spam.csv

In [ ]:
import pandas as pd
df = pd.read_csv('spam.csv' , encoding = 'utf-8' , sep = "\t")
df.head()

In [ ]:
df.shape

In [ ]:
round(df['Type'].value_counts(normalize = True)*100,2)

In [ ]:
df["length"] = df["Message"].apply(len)

In [ ]:
df.head()

### Data Visulization

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
# df['length'].plot.hist(bins = 50)
df['length'].plot(kind = 'hist' , bins = 50)

In [ ]:
df['Type'].unique()

In [ ]:
df[df['Type']== 'ham']['length'].plot(kind = 'hist' , bins = 50 , alpha = 0.5 , color = 'green' , label = 'ham messages')
df[df['Type']== 'spam']['length'].plot(kind = 'hist' , bins = 50 , alpha = 0.5 , color = 'blue' , label = 'spam messages')

In [ ]:
for messages in df['Message']:
  print(messages)

Problems in message col:

 - Not a fix format
 - Spelling problems  - Because spelling problem , we need advanced models e.g. Glove , Elmo , BERT , Roberta , Good amount of dictionary, Spelling correction models.
 - Special characters
 - Upper case issues

In [ ]:
import string
mess = "This is sample messaging!@.."

In [ ]:
mess

In [ ]:
nonpunc = ''.join([char for char in mess if char not in string.punctuation])

In [ ]:
nonpunc

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stopwords.words('english')

In [ ]:
clean_mess = [word for word in nonpunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
clean_mess

In [ ]:
# Text Process
def text_preprocess(mess):
  nonpunc = ''.join([char for char in mess if char not in string.punctuation])
  clean_mess = [word for word in nonpunc.split() if word.lower() not in stopwords.words('english')]
  return clean_mess

In [ ]:
df['word_processed'] = df['Message'].apply(text_preprocess)

In [ ]:
df['word_processed']

In [ ]:
df.head()

In [ ]:
df['word_processed'].value_counts()[:10].plot(kind = 'bar')

In [ ]:
#How to find most common words in SPAM messages
df[df['Type']=='spam']['word_processed'].value_counts()[:10]

In [ ]:
df[df['Type']=='ham']['word_processed'].value_counts()[:10]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bow_transformer = CountVectorizer(analyzer = text_preprocess).fit(df['Message'])

In [ ]:
len(bow_transformer.vocabulary_)

In [ ]:
df_03 = df['Message'][3]

In [ ]:
df_03

In [ ]:
bow_03 = bow_transformer.transform([df_03])
print(df_03)
print("============")
print(bow_03)
print("===============")
print(bow_03.shape)

In [ ]:
print(df_03)
print("============")
print(bow_transformer.get_feature_names_out()[873])
print("============")
print(bow_transformer.get_feature_names_out()[2320])

In [ ]:
df_bow = bow_transformer.transform(df['Message'])

In [ ]:
df_bow.shape

In [ ]:
print(df_bow.nnz)

In [ ]:
sparsity = round((100.0*df_bow.nnz/(df_bow.shape[0]*df_bow.shape[1])),3)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
tfidf_transformer = TfidfTransformer().fit(df_bow)

In [ ]:
tfidf_03 = tfidf_transformer.transform(bow_03)

In [ ]:
print(tfidf_03)

In [ ]:
print(tfidf_transformer.idf_[bow_transformer.vocabulary_['U']])
print(tfidf_transformer.idf_[bow_transformer.vocabulary_['say']])
print(tfidf_transformer.idf_[bow_transformer.vocabulary_['already']])
print(tfidf_transformer.idf_[bow_transformer.vocabulary_['early']])

In [ ]:
df_tfidf = tfidf_transformer.transform(df_bow)

In [ ]:
print(df_tfidf.shape)

### TRain the model

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
spam_detect_model = MultinomialNB().fit(df_tfidf , df['Type'])

In [ ]:
print("predicted-" , spam_detect_model.predict(tfidf_03)[0])
print("actual-" , df.Type[3])

In [ ]:
all_predictions = spam_detect_model.predict(df_tfidf)

In [ ]:
all_predictions

In [ ]:
from sklearn.model_selection import train_test_split

sms_train , sms_test , label_train , label_test = train_test_split(df['Message'] , df['Type'] , test_size = 0.4)

In [ ]:
print("Shape of SMS Train Data" , sms_train.shape)
print("Shape of SMS Label Train Data" , label_train.shape)
print("Shape of SMS Test Data" , sms_test.shape)
print("Shape of SMS label test Data" , label_test.shape)


In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipeline = Pipeline([
    ('bow' , CountVectorizer(analyzer = text_preprocess)),
    ('tfidf' , TfidfTransformer()),
    ('classifier' , MultinomialNB())
])

In [ ]:
pipeline.fit(sms_train , label_train)

In [ ]:
predictions = pipeline.predict(sms_test)

In [ ]:
next_text = ["During our last conversation we agreed to deliver goods" ,
             "you have been selected to receive 10000 cash prize"]

predictions_01 = pipeline.predict(next_text)
print(predictions_01)